# Libs e Dados

In [1]:
# change directory
%cd ..

/home/victor/text_mining_medium


In [2]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import load_npz
from src.Text_Mining import TopicClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

In [3]:
data = pd.read_parquet("data/articles_processed.parquet")
tf    = load_npz('models/tf_unigram_articles.npz')
tfidf = load_npz('models/tfidf_unigram_articles.npz')
vocabulary = pd.read_csv("models/vocabulary_unigram_articles.csv").ngrams
doc_top_wals = np.load(f'models/doc_topics_wals_unigram_articles.npy').astype('float32')
word_top_wals = np.load(f'models/word_topics_wals_unigram_articles.npy').astype('float32')

assert tf.shape == tfidf.shape, 'Rever BOW'
assert tf.shape[0] == data.shape[0], 'Rever BOW'
assert tf.shape[0] == doc_top_wals.shape[0], 'Rever Modelagem de Tópicos'
assert word_top_wals.shape[1] == doc_top_wals.shape[1], 'Rever Modelagem de Tópicos'

In [4]:
data.head()

,title,text,date,category,subcategory,link
0,lula diz esta lascado que ainda forca cabo ele...,com possibilidade uma condenacao impedir candi...,2017-09-10,poder,None,http://www1.folha.uol.com.br/poder/2017/10/192...
1,decidi escrava mulheres sofrem diz cantora oum...,para oumou sangare cantora ativista malinesa s...,2017-09-10,ilustrada,None,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,tres reportagens folha ganham premio petrobras...,tres reportagens folha vencedoras premio petro...,2017-09-10,poder,None,http://www1.folha.uol.com.br/poder/2017/10/192...
3,filme star wars os ultimos jedi ganha trailer ...,a disney divulgou noite desta segundafeira no...,2017-09-10,ilustrada,None,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,cbss inicia acordos fintechs quer credito dig...,o cbss banco holding elopar socios bradesco ba...,2017-09-10,mercado,None,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [5]:
data.shape

(167053, 6)

# Verificando Vocabulário

**Palavras Mais Comuns**

In [6]:
words_df = pd.DataFrame({"word":vocabulary.values, 'freq': np.asarray(tf.sum(axis=0))[0]}
                       ).sort_values('freq', ascending=False)
words_df.head(20).T

,4050,22800,47708,61435,33597,61982,24268,2331,4025,9741,54102,43052,23138,46871,49744,50699,50014,64303,47467,14347
word,anos,disse,no,sao,governo,segundo,ele,ainda,ano,brasil,presidente,mais,diz,na,pais,paulo,para,sua,nesta,com
freq,148358,138855,138844,138626,131625,128596,125899,125491,124475,120982,118032,117031,116539,112803,111588,109385,105829,93992,91116,88436


**Top 250 words**

In [7]:
'  '.join(words_df.iloc[:250].word)

'anos  disse  no  sao  governo  segundo  ele  ainda  ano  brasil  presidente  mais  diz  na  pais  paulo  para  sua  nesta  com  pode  pessoas  contra  seu  dois  mas  apos  dia  isso  foi  da  milhoes  afirmou  do  rio  desde  tem  fazer  maior  parte  tres  caso  casa  outros  por  mil  menos  ser  mundo  apenas  como  federal  acordo  estado  porque  ja  todos  hoje  alem  empresa  tempo  onde  antes  dilma  vez  folha  agora  ela  grande  eu  ha  tambem  primeiro  cidade  deve  neste  esta  publico  dias  politica  nos  mesmo  outro  grupo  duas  nova  durante  seus  bem  primeira  nacional  afirma  empresas  semana  novo  passado  cada  temer  trabalho  assim  ao  meio  vida  mercado  ate  bilhoes  ministro  melhor  policia  fim  processo  justica  cerca  conta  estados  eles  todo  quem  quando  forma  brasileiro  final  ministerio  ter  depois  sido  us  outras  sempre  quatro  meses  eua  decisao  relacao  podem  crise  alguns  momento  camara  fez  regiao  entao  muito  dinhei

# Modelos de Tópicos

## Resumo dos modelos de tópicos

Criando resumo de tópicos por palavras por weight e relevância

In [8]:
%%time
topwords_relevance_df_wals_topic, top5_words_relevance_wals_topic = TopicClustering.view_topics(word_top_wals, vocabulary, relevance=True, n_top_words=50)
topwords_weight_df_wals_topic, top5_words_weight_wals_topic       = TopicClustering.view_topics(word_top_wals, vocabulary, relevance=False, n_top_words=50)

CPU times: user 2.8 s, sys: 47.1 ms, total: 2.84 s
Wall time: 2.84 s


**Os dataframes com as top 50 palavras para cada tópico**

In [9]:
topwords_relevance_df_wals_topic

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,dados,fato,diretor,opiniao,negociacoes,defesa,servicos,cunha,deputados,europa,...,federal,jogar,jornal,estados,neste,senador,pontos,equipe,criancas,pesquisa
1,queda,entre,uruguai,militar,acordo,foram,poder,camara,deputado,impostos,...,programa,eu,york,unidos,blocos,aecio,estados,titulo,publico,datafolha
2,afeta,menos,argentina,bola,plano,ensino,primeiro,deputados,camara,fiscal,...,numero,disse,times,sancoes,carnaval,neves,americanos,com,crianca,ate
3,coordenacao,parece,paraguai,vezes,trump,prova,recursos,plenario,denuncia,quartafeira,...,formacao,estar,new,russia,processo,senado,unidos,vitoria,filhos,alckmin
4,sua,policial,pessoal,pm,proposta,enem,postos,pmdbrj,parlamentar,administrativo,...,alem,estatal,decisao,bombeiros,festa,vezes,americana,branco,familia,pesquisas
5,aviao,janeiro,terceiro,politica,bilhoes,liberdade,setor,senado,janot,mundo,...,senado,petrobras,opiniao,porto,hora,muitas,real,resultado,vamos,geraldo
6,troya,fim,argentino,jogo,bilhao,estudantes,ibge,eduardo,rodrigo,cade,...,constituicao,australia,geral,ira,palacio,renda,la,venceu,trabalho,psdb
7,estudo,policia,river,estava,nuclear,joao,dados,afastamento,procuradoriageral,servidores,...,politicos,dupla,mendes,longo,bloco,senadores,eua,preco,deve,turno
8,klimpel,policiais,levou,diversas,aprovacao,dias,aos,parlamentares,inquerito,conselho,...,ao,estou,gilmar,camara,apresentar,perfil,americano,exposicao,infantil,ficou
9,reportagem,talvez,plate,futebol,donald,silva,republica,senadores,gabinete,aumento,...,supremo,deu,nacional,sociedade,sabado,psdbmg,madri,tel,pai,agua


**Visualização as séries de top5 palavras para cada tópico: permite uma visualização muito mais rápida**

In [10]:
df_words5_topics = pd.DataFrame({"topic_words_weight": top5_words_weight_wals_topic,
                                 "topic_words_relevance": top5_words_relevance_wals_topic})

In [11]:
df_words5_topics

,topic_words_weight,topic_words_relevance
0,sua | dados | tambem | queda | reportagem,dados | queda | afeta | coordenacao | sua
1,fato | menos | fim | entre | janeiro,fato | entre | menos | parece | policial
2,diretor | foi | para | dois | paulo,diretor | uruguai | argentina | paraguai | pes...
3,militar | opiniao | vezes | politica | grande,opiniao | militar | bola | vezes | pm
4,acordo | plano | bilhoes | proposta | seu,negociacoes | acordo | plano | trump | proposta
...,...,...
295,senador | vezes | brasileiros | muitas | senado,senador | aecio | neves | senado | vezes
296,pontos | estados | unidos | eua | americana,pontos | estados | americanos | unidos | ameri...
297,com | equipe | resultado | dia | titulo,equipe | titulo | com | vitoria | branco
298,publico | criancas | deve | familia | trabalho,criancas | publico | crianca | filhos | familia


# Modelos de Clusterização Rígida

**Carregandos os dados da clusterização**

In [12]:
clustering_extensive_50  = np.load('models/clusters_extensive_50_outputs.npz')
clustering_intensive_200 = np.load('models/clusters_intensive_200_outputs.npz')
data['cluster_ext_50']  = clustering_extensive_50['index']
data['cluster_int_200'] = clustering_intensive_200['index']

## Resumo de Clusterização
Da mesma maneira que foi realizado para os tópicos, o resumos dos clusters se fará por palavras.

### Clusterização Extensiva

In [13]:
%%time
word_cluster_ext_50 = TopicClustering.get_word_cluster_matrix(data.cluster_ext_50, tf).astype('float32')
topwords_relevance_df_ext_50, top5_words_relevance_ext_50 = TopicClustering.view_topics(word_cluster_ext_50, vocabulary, relevance=True, n_top_words=50)
topwords_weight_df_ext_50, top5_words_weight_ext_50       = TopicClustering.view_topics(word_cluster_ext_50, vocabulary, relevance=False, n_top_words=50)

CPU times: user 826 ms, sys: 11.2 ms, total: 837 ms
Wall time: 836 ms


In [14]:
df_words5_clusters_ext_50 = pd.DataFrame({'n_instances': data.cluster_ext_50.value_counts(),
                                          "cluster_words_weight": top5_words_weight_ext_50, 
                                          "cluster_words_relevance": top5_words_relevance_ext_50})

**Série das top5 dos clusters**

In [15]:
df_words5_clusters_ext_50

,n_instances,cluster_words_weight,cluster_words_relevance
0,2817,empresa | apple | google | disse | us,apple | google | iphone | android | usuarios
1,3744,governo | presidente | pais | disse | venezuela,maduro | venezuela | caracas | coreia | venezu...
2,1976,brasil | para | sua | pais | participacao,traduzem | obedece | contemporaneo | refletir ...
3,4905,time | selecao | contra | clube | jogador,jogador | neymar | selecao | jogadores | palme...
4,434,voce | hoje | lua | seu | pode,jul | fev | abr | nov | lua
5,2868,brasil | folha | leitores | no | para,enviando | envie | leitoruolcombr | leitores |...
6,964,sportv | espn | brasil | nba | copa,sportv | espn | nba | warriors | sports
7,3737,anos | pode | saude | mais | diz,virus | zika | cientistas | pesquisadores | mi...
8,4780,governo | ano | bilhoes | presidente | anos,governo | bilhoes | imposto | receita | previd...
9,1967,folha | brasil | sao | paulo | social,empreendedor | inscricoes | pags | empreendedo...


### Clusterização Intensiva

In [16]:
%%time
word_cluster_int_200 = TopicClustering.get_word_cluster_matrix(data.cluster_int_200, tf).astype('float32')
topwords_relevance_df_int_200, top5_words_relevance_int_200 = TopicClustering.view_topics(word_cluster_int_200, vocabulary, relevance=True, n_top_words=50)
topwords_weight_df_int_200, top5_words_weight_int_200       = TopicClustering.view_topics(word_cluster_int_200, vocabulary, relevance=False, n_top_words=50)

CPU times: user 2.24 s, sys: 83.6 ms, total: 2.32 s
Wall time: 2.32 s


In [17]:
df_words5_clusters_int_200 = pd.DataFrame({'n_instances': data.cluster_int_200.value_counts(),
                                           "cluster_words_weight": top5_words_weight_int_200, 
                                           "cluster_words_relevance": top5_words_relevance_int_200})

**Série das top5 dos clusters**

In [18]:
df_words5_clusters_int_200

,n_instances,cluster_words_weight,cluster_words_relevance
0,766,cunha | camara | presidente | conselho | processo,cunha | cassacao | etica | pinato | camara
1,868,sp | sao | paulo | brasil | folha,sp | leitorgrupofolhacombr | paulo | sao | lei...
2,840,temer | presidente | joesley | jbs | ele,joesley | jbs | loures | batista | saud
3,757,indice | china | economia | pontos | crescimento,indice | yuan | xangai | china | fed
4,737,indigenas | governo | area | terras | segundo,indigenas | funai | indios | terras | desmatam...
...,...,...,...
195,422,mins | hamilton | gp | massa | felipe,mins | vettel | hamilton | nasr | rosberg
196,867,ano | queda | trimestre | ibge | mes,ibge | trimestre | queda | desemprego | compar...
197,839,odebrecht | empresa | petrobras | lava | jato,leniencia | odebrecht | cgu | cartel | empreit...
198,218,dia | paulo | personagem | vida | sempre,troya | klimpel | preparese | teles | seragusa


**Visualizando parte do dataframe das top50**

In [26]:
topwords_relevance_df_int_200.iloc[:10, :7]

,0,1,2,3,4,5,6
0,cunha,sp,joesley,indice,indigenas,inbev,colunaobituariogrupofolhacombr
1,cassacao,leitorgrupofolhacombr,jbs,yuan,funai,sabmiller,anuncios
2,etica,paulo,loures,xangai,indios,ab,missas
3,pinato,sao,batista,china,terras,yahoo,mortes
4,camara,leitores,saud,fed,desmatamento,kroton,netos
5,conselho,enviando,janot,shenzhen,indigena,estacio,veja
6,peemedebista,envie,temer,bce,amazonia,us,gostava
7,deputado,colunistas,pgr,csi,renca,buffett,morreu
8,maranhao,acontecimentos,miller,ssec,demarcacao,transacao,filhos
9,pmdbrj,relevantes,delacao,pmi,hectares,aquisicao,sobrinhos


**Uma amostra das top5**

In [21]:
df_words5_clusters_int_200.sample(10)

,n_instances,cluster_words_weight,cluster_words_relevance
60,824,presidente | camara | temer | governo | deputados,maia | centrao | camara | deputados | rosso
84,1335,pessoas | samarco | segundo | ainda | disse,samarco | barragem | lama | terremoto | barragens
92,571,trabalho | trabalhadores | governo | empresas ...,esocial | terceirizacao | trabalhador | empreg...
45,1322,saude | medicos | hospital | diz | anos,saude | medicos | sus | hospital | pacientes
168,1212,dilma | presidente | contra | impeachment | ma...,manifestantes | mbl | ato | protesto | manifes...
50,1598,policia | segundo | ele | disse | sao,policia | bombeiros | corpo | incendio | iml
71,434,turquia | erdogan | governo | turco | pais,erdogan | turquia | turco | gulen | recep
113,1158,show | banda | rock | rio | festival,rock | banda | show | palco | shows
87,826,partido | governo | pais | presidente | eleicoes,macron | pen | fillon | le | marine
140,527,cerveja | vinho | tel | sao | vinhos,cerveja | vinhos | vinho | ml | cervejas


# Diagnóstico de Categorias

## 1. Salvando dataframe da clusterização extensiva para rotulação de assuntos

In [32]:
df_words5_clusters_ext_50.reset_index(
                                      ).rename(columns={'index':'cluster'}
                                               ).to_csv('reports/extensive_clusters.csv',index=False)

## 2. Rotulação em Assuntos Principais e Secundários 

## 3. Avaliação dos Rótulos

**Lendo o arquivo após rotulação manual**

O arquivo está disponível no [link](https://docs.google.com/spreadsheets/d/12lQq2AH9EUMTp-afz4l_MzsAEwBIhcZQwtAP4HzOu74/edit#gid=1235989610)

In [34]:
ext_clusters = pd.read_csv('reports/extensive_clusters.csv')

In [44]:
# fix
ext_clusters = ext_clusters.applymap(lambda x: 'indefinido' if x=='indefinido ' else x)

**Número de assuntos distintos**

In [45]:
ext_clusters.nunique()[-2:]

assunto_princ    25
assunto_sec      35
dtype: int64

**Assuntos Principais Distintos**

In [49]:
print(pd.Series(ext_clusters.assunto_princ.unique()).sort_values().values)

['acidentes' 'arte' 'brasil' 'economia' 'educacao' 'espanhol' 'esportes'
 'gastronomia' 'governo ' 'indefinido' 'ingles' 'lava jato' 'literatura'
 'midias' 'moda' 'mundo ' 'musica' 'policia' 'politica' 'sao paulo '
 'saude' 'sindicatos' 'tecnologia' 'transportes' 'tv e cinema']


In [53]:
resum_assuntos = ext_clusters.groupby("assunto_princ"
                                          ).agg({'n_instances':'sum',
                                                 'cluster': 'nunique',
                                                 'assunto_sec':'nunique'}
                                                ).sort_values("n_instances", 
                                                              ascending=False)

resum_assuntos.columns = ["n_instances", 'n_cluster', 'n_assunto_secundario']
resum_assuntos

,n_instances,n_cluster,n_assunto_secundario
assunto_princ,,,
politica,23266,5,4
indefinido,21148,6,2
esportes,19368,7,6
sao paulo,17848,1,1
mundo,13829,5,5
economia,9688,3,3
midias,7101,4,3
tv e cinema,6761,2,2
lava jato,5898,1,1


## 4. Avaliando um Categoria Individualmente: Esportes

### Definindo todos os clusters da clusterização extensiva que são de Esportes

In [54]:
ext_clusters_esportes = [3, 6, 14, 16, 24, 40, 45]

In [57]:
ext_clusters.loc[ext_clusters_esportes]

,cluster,n_instances,cluster_words_weight,cluster_words_relevance,assunto_princ,assunto_sec
3,3,4905,time | selecao | contra | clube | jogador,jogador | neymar | selecao | jogadores | palme...,esportes,futebol
6,6,964,sportv | espn | brasil | nba | copa,sportv | espn | nba | warriors | sports,esportes,midias
14,14,1539,ano | ranking | brasileiro | final | mundial,gp | sets | bellucci | djokovic | hamilton,esportes,geral
16,16,2386,gol | time | jogo | pontos | com,min | gol | rodada | placar | gols,esportes,futebol
24,24,3973,futebol | cbf | fifa | presidente | copa,cbf | fifa | futebol | nero | blatter,esportes,federacoes
40,40,1668,sao | paulo | time | contra | clube,ceni | muricy | morumbi | libertadores | bauza,esportes,spfc
45,45,3933,rio | jogos | brasil | mundial | atletas,atletas | jogos | medalhas | medalha | olimpico,esportes,olimpiadas


### Consulta na clusterização intensiva

**Consultando os clusters de clusterização intensiva para os documentos cujo os clusters extensivos sejam de esportes**

In [63]:
contagem_clusters_int_esportes = data.query('cluster_ext_50 in @ext_clusters_esportes').cluster_int_200.value_counts()

In [95]:
esportes_ext_data = data.query("cluster_ext_50 in @ext_clusters_esportes")
esportes_intensive = pd.DataFrame({'% de noticias':esportes_ext_data.cluster_int_200.value_counts(normalize=True)*100,
                                   'n de noticias':esportes_ext_data.cluster_int_200.value_counts()}
                                   ).join(df_words5_clusters_int_200)
futebol_intensive.head(11)

,% de noticias,n de noticias,cluster,n_instances,cluster_words_weight,cluster_words_relevance,Assunto Princ.,Assunto Sec.
177,19.082569,936,177,1489,clube | sao | paulo | futebol | corinthians,clube | jogador | aidar | palmeiras | corinthians,esportes,futebol paulista
146,18.695209,917,146,1122,barcelona | neymar | jogador | messi | clube,barcelona | messi | neymar | madrid | cristiano,esportes,futebol espanhol
83,17.166157,842,83,1108,selecao | copa | contra | dunga | brasil,dunga | selecao | eliminatorias | copa | neymar,esportes,selecao brasileira
182,15.290520,750,182,858,palmeiras | contra | time | santos | jogo,palmeiras | dudu | arouca | cuca | robinho,esportes,futebol
52,10.988787,539,52,755,corinthians | time | tite | contra | equipe,corinthians | tite | jadson | uendel | guerrero,esportes,corinthians
138,8.073394,396,138,461,futebol | time | brasil | bola | jogadores,passes | bola | meiocampo | treinadores | joga...,esportes,futebol tatica
124,1.855250,91,124,742,pontos | time | brasileiro | rodada | campeonato,rodada | coritiba | avai | gremio | figueirense,esportes,brasileirao
46,1.773700,87,46,1626,estadio | sao | torcedores | futebol | jogo,torcedores | estadio | torcida | palmeiras | c...,esportes,torcida
114,1.508665,74,114,709,sao | paulo | libertadores | time | corinthians,libertadores | river | amarilla | juniors | plate,esportes,copa libertadores
58,1.100917,54,58,925,selecao | brasil | rio | jogo | jogos,volei | selecao | jogadoras | bernardinho | ba...,esportes,volei


**Vê se uma maior Especificação e qualidade na clusterização intensiva**

Segundo o framework, não se faz rotulação da base completa de clusterização intensiva, mas especialmente nesse exemplo será feito por motivos didáticos

O arquivo de rotulação dos clusters intensivos está disponível no [link](https://docs.google.com/spreadsheets/d/1QLRhLSokSrnY7F_AoaHwUSufip1IlJneTXws8Vv1bFE/edit#gid=579318358)

**Lendo após rotulação**

In [74]:
int_clusters = pd.read_csv('reports/intensive_clusters.csv')

fazendo a mesma consulta, agr com os assuntos rotulados

In [96]:
esportes_ext_data = data.query("cluster_ext_50 in @ext_clusters_esportes")
esportes_intensive = pd.DataFrame({'% de noticias':esportes_ext_data.cluster_int_200.value_counts(normalize=True)*100,
                                   'n de noticias':esportes_ext_data.cluster_int_200.value_counts()}
                                   ).join(int_clusters)
futebol_intensive.head(11)

,% de noticias,n de noticias,cluster,n_instances,cluster_words_weight,cluster_words_relevance,Assunto Princ.,Assunto Sec.
177,19.082569,936,177,1489,clube | sao | paulo | futebol | corinthians,clube | jogador | aidar | palmeiras | corinthians,esportes,futebol paulista
146,18.695209,917,146,1122,barcelona | neymar | jogador | messi | clube,barcelona | messi | neymar | madrid | cristiano,esportes,futebol espanhol
83,17.166157,842,83,1108,selecao | copa | contra | dunga | brasil,dunga | selecao | eliminatorias | copa | neymar,esportes,selecao brasileira
182,15.290520,750,182,858,palmeiras | contra | time | santos | jogo,palmeiras | dudu | arouca | cuca | robinho,esportes,futebol
52,10.988787,539,52,755,corinthians | time | tite | contra | equipe,corinthians | tite | jadson | uendel | guerrero,esportes,corinthians
138,8.073394,396,138,461,futebol | time | brasil | bola | jogadores,passes | bola | meiocampo | treinadores | joga...,esportes,futebol tatica
124,1.855250,91,124,742,pontos | time | brasileiro | rodada | campeonato,rodada | coritiba | avai | gremio | figueirense,esportes,brasileirao
46,1.773700,87,46,1626,estadio | sao | torcedores | futebol | jogo,torcedores | estadio | torcida | palmeiras | c...,esportes,torcida
114,1.508665,74,114,709,sao | paulo | libertadores | time | corinthians,libertadores | river | amarilla | juniors | plate,esportes,copa libertadores
58,1.100917,54,58,925,selecao | brasil | rio | jogo | jogos,volei | selecao | jogadoras | bernardinho | ba...,esportes,volei


**Seguindo a mesma ideia para um assunto secundário específico: futebol (cluster extensivo 3)**

In [97]:
# relembrando clusters exensivos de esportes
ext_clusters.loc[ext_clusters_esportes]

,cluster,n_instances,cluster_words_weight,cluster_words_relevance,assunto_princ,assunto_sec
3,3,4905,time | selecao | contra | clube | jogador,jogador | neymar | selecao | jogadores | palme...,esportes,futebol
6,6,964,sportv | espn | brasil | nba | copa,sportv | espn | nba | warriors | sports,esportes,midias
14,14,1539,ano | ranking | brasileiro | final | mundial,gp | sets | bellucci | djokovic | hamilton,esportes,geral
16,16,2386,gol | time | jogo | pontos | com,min | gol | rodada | placar | gols,esportes,futebol
24,24,3973,futebol | cbf | fifa | presidente | copa,cbf | fifa | futebol | nero | blatter,esportes,federacoes
40,40,1668,sao | paulo | time | contra | clube,ceni | muricy | morumbi | libertadores | bauza,esportes,spfc
45,45,3933,rio | jogos | brasil | mundial | atletas,atletas | jogos | medalhas | medalha | olimpico,esportes,olimpiadas


In [93]:
futebol_data = data.query("cluster_ext_50 == 3")
futebol_intensive = pd.DataFrame({'% de noticias':futebol_data.cluster_int_200.value_counts(normalize=True)*100,
                                  'n de noticias':futebol_data.cluster_int_200.value_counts()}
                                  ).join(int_clusters)
futebol_intensive.head(9)

,% de noticias,n de noticias,cluster,n_instances,cluster_words_weight,cluster_words_relevance,Assunto Princ.,Assunto Sec.
177,19.082569,936,177,1489,clube | sao | paulo | futebol | corinthians,clube | jogador | aidar | palmeiras | corinthians,esportes,futebol paulista
146,18.695209,917,146,1122,barcelona | neymar | jogador | messi | clube,barcelona | messi | neymar | madrid | cristiano,esportes,futebol espanhol
83,17.166157,842,83,1108,selecao | copa | contra | dunga | brasil,dunga | selecao | eliminatorias | copa | neymar,esportes,selecao brasileira
182,15.290520,750,182,858,palmeiras | contra | time | santos | jogo,palmeiras | dudu | arouca | cuca | robinho,esportes,futebol
52,10.988787,539,52,755,corinthians | time | tite | contra | equipe,corinthians | tite | jadson | uendel | guerrero,esportes,corinthians
138,8.073394,396,138,461,futebol | time | brasil | bola | jogadores,passes | bola | meiocampo | treinadores | joga...,esportes,futebol tatica
124,1.855250,91,124,742,pontos | time | brasileiro | rodada | campeonato,rodada | coritiba | avai | gremio | figueirense,esportes,brasileirao
46,1.773700,87,46,1626,estadio | sao | torcedores | futebol | jogo,torcedores | estadio | torcida | palmeiras | c...,esportes,torcida
114,1.508665,74,114,709,sao | paulo | libertadores | time | corinthians,libertadores | river | amarilla | juniors | plate,esportes,copa libertadores


In [ ]:
futebol_data = data.query("cluster_wals_50==3")
futebol_intensive = pd.DataFrame({'% de noticias':futebol_data.cluster_wals_200.value_counts(normalize=True)*100,
              'n de noticias':futebol_data.cluster_wals_200.value_counts()}
            ).join(art.iloc[:,:-1]).head(20)

**Vê se uma maior especificação do tema futebol, o que pode se tornar categorias distintas e hierarquias, aqui entra a parte artística do framework**

## 5.Validação: visualizando amostras de um cluster intensivo 

Para criar categorias, utilize sempre os clusters intensivos, por ser de maior qualidade e especificidade

Vendo uma amostra específica de futebol. Vou pegar o da Seleção Brasileira (cluster intensivo 83), que joga hoje, 04/06/2021, dia da publicação do artigo.

Obs: minha predição diz que hoje tem gol do gabigol.

In [103]:
data.query("cluster_int_200==83").sample(10).iloc[:, [0,1, -1,-2]]

,title,text,cluster_int_200,cluster_ext_50
131083,selecao joga evitar vexame por melhor caminho ...,o jogo deste domingo selecao brasileira contr...,83,24
131531,depois liga pedir perdao diz neymar jogador o ...,um atrativos a partida brasil colombia nesta q...,83,3
127616,aniversario um ano a alemanha vira piada int...,o aniversario um ano derrota selecao brasileir...,83,24
129607,medico selecao diz nao conseguiu identificar c...,o medico selecao brasileira rodrigo lasmar dis...,83,3
24284,em retaliacao tite fecha treino tatico selecao...,tite fechou treino tatico deste domingo itaqu...,83,3
126530,fifa nega recurso cbf liberar neymar punicao e...,a comissao disciplinar fifa negou recurso cbf ...,83,24
73796,cbf divulga preconvocados dunga copa america n...,a cbf promete divulgar nesta sexta jogadores...,83,3
109835,selecao chega fortaleza jogadores vao direto f...,a selecao brasileira esta fortaleza a partida ...,83,3
67469,um estorvo chamado selecao,o contra equador um primeiro tempo razoavel ...,83,24
110769,philippe coutinho cortado dunga convoca kaka e...,o tecnico dunga convocou noite deste domingo ...,83,3


Vê se a altíssima qualidade da clusterização intensiva. Pra se ter uma noção, esportes representam 21 clusters e 18 assuntos secundários distintos.

In [110]:
resum_assuntos = int_clusters.groupby('Assunto Princ.'
                                          ).agg({'n_instances':'sum',
                                                 'cluster': 'nunique',
                                                 'Assunto Sec.':'nunique'}
                                                ).sort_values("n_instances", 
                                                              ascending=False)

resum_assuntos.columns = ["n_instances", 'n_cluster', 'n_assunto_secundario']
resum_assuntos.head(30)

,n_instances,n_cluster,n_assunto_secundario
Assunto Princ.,,,
politica,24266,26,18
esportes,17862,21,18
mundo,12682,16,14
economia,9616,10,9
midia,7922,10,5
indefinido,7081,11,4
crimes,6012,7,5
tv e cinema,5113,4,4
saude,4273,4,4
